# Imports

- Numpy
- os
- tensorflow
- time
- Image

In [1]:
from numpy.random import seed
seed(888)
import tensorflow.compat.v1 as tf
tf.set_random_seed(404)

import os
import numpy as np

from time import strftime
from PIL import Image

# Constant Variables

- X Train
- X Test
- Y Train
- Y Test


In [2]:
X_TRAIN_PATH = 'data/digit_xtrain.csv'
X_TEST_PATH = 'data/digit_xtest.csv'
Y_TRAIN_PATH = 'data/digit_ytrain.csv'
Y_TEST_PATH = 'data/digit_ytest.csv'

LOGGING_PATH = 'hw_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

# Data Gathering

- Testing the data
- CPU Load Times
- Shape the data

In [3]:
%%time

y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 49.6 ms, sys: 1.84 ms, total: 51.4 ms
Wall time: 53.3 ms


In [4]:
y_train_all.shape

(60000,)

In [5]:
%%time

x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

CPU times: user 11.9 s, sys: 401 ms, total: 12.3 s
Wall time: 12.3 s


In [6]:
x_train_all.shape

(60000, 784)

# Data Explore

In [7]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [8]:
x_test.shape

(10000, 784)

In [9]:
y_test.shape

(10000,)

In [10]:
y_train_all[:5]

array([5, 0, 4, 1, 9])

# Data Preprocessing

- Convert target values to one-hot encoding
- Create validation dataset from training data

In [11]:
# Re-scale
x_train_all, x_test = x_train_all / 255.0, x_test / 255.0

### Convert target values to one-hot encoding

In [12]:
values = y_train_all[:5]
np.eye(10)[values]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [13]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [14]:
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [15]:
values

array([5, 0, 4, 1, 9])

In [16]:
values[4]

9

In [17]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]
y_train_all.shape


(60000, 10)

In [18]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

# Create validation dataset from training data

In [19]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [20]:
x_train.shape

(50000, 784)

In [21]:
x_val.shape

(10000, 784)

# Setup Tensorflow Graph

- Neural Network Architecture
- Hyperparameters

In [22]:
tf.compat.v1.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X')
Y = tf.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels')

In [23]:
nr_epochs = 50
learning_rate = 1e-3

n_hidden1 = 512
n_hidden2 = 64

In [24]:
def setup_layer(input, weight_dim, bias_dim, name):
    
    with tf.name_scope(name):
        initial_w = tf.truncated_normal(shape=weight_dim, stddev=0.1, seed=42)
        w = tf.Variable(initial_value=initial_w, name='W')

        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b
        
        if name=='out':
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)
        
        tf.summary.histogram('weights', w)
        tf.summary.histogram('biases', b)
        
        return layer_out

In [25]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1], 
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, rate=0.8, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2], 
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES], 
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

# Tensorboard Setup

In [26]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H:%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else:
    print('Successfully created directories!')

Successfully created directories!


# Loss, Optimisation & Metrics

- Defining Loss Function
- Defining Optimizer
- Accuracy Metric
- Check Input Images in Tensorboard


In [27]:
with tf.name_scope('loss_calc'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output))

In [28]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train_step = optimizer.minimize(loss)

In [29]:
with tf.name_scope('accuracy_calc'):
    correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [30]:
with tf.name_scope('performance'):
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.scalar('cost', loss)

In [31]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1, 28, 28, 1])
    tf.summary.image('image_input', x_image, max_outputs=4)

# Run Session

- Setup Filewriter and Merge Summaries
- Initialise all the variables
- Batching the Data
- Training Loop

In [32]:
sess = tf.Session()

In [33]:
merged_summary = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph)

validation_writer = tf.summary.FileWriter(directory + '/validation')

In [34]:
init = tf.global_variables_initializer()
sess.run(init)

2022-05-21 21:42:34.570985: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [35]:
size_of_batch = 1000

In [36]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [37]:
def next_batch(batch_size, data, labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end], labels[start:end]

In [38]:
for epoch in range(nr_epochs):
    
    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)
        
        feed_dictionary = {X:batch_x, Y:batch_y}
        
        sess.run(train_step, feed_dict=feed_dictionary)
        
    
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)
        
    train_writer.add_summary(s, epoch)
    
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')
    
    # ================== Validation ======================
    
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done training!')

Epoch 0 	| Training Accuracy = 0.7620000243186951
Epoch 1 	| Training Accuracy = 0.8159999847412109
Epoch 2 	| Training Accuracy = 0.8389999866485596
Epoch 3 	| Training Accuracy = 0.8399999737739563
Epoch 4 	| Training Accuracy = 0.8500000238418579
Epoch 5 	| Training Accuracy = 0.8529999852180481
Epoch 6 	| Training Accuracy = 0.8529999852180481
Epoch 7 	| Training Accuracy = 0.8600000143051147
Epoch 8 	| Training Accuracy = 0.949999988079071
Epoch 9 	| Training Accuracy = 0.9490000009536743
Epoch 10 	| Training Accuracy = 0.9580000042915344
Epoch 11 	| Training Accuracy = 0.9599999785423279
Epoch 12 	| Training Accuracy = 0.9610000252723694
Epoch 13 	| Training Accuracy = 0.9610000252723694
Epoch 14 	| Training Accuracy = 0.9679999947547913
Epoch 15 	| Training Accuracy = 0.9679999947547913
Epoch 16 	| Training Accuracy = 0.9710000157356262
Epoch 17 	| Training Accuracy = 0.9679999947547913
Epoch 18 	| Training Accuracy = 0.9639999866485596
Epoch 19 	| Training Accuracy = 0.97299998

# Make a Prediction

In [39]:
img = Image.open('data/test_img.png')
img

In [40]:
bw = img.convert('L')
img_array = np.invert(bw)

img_array.shape

(28, 28)

In [41]:
test_img = img_array.ravel()
test_img.shape

(784,)

In [42]:
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X:[test_img]})
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evaluation

In [43]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 95.64%


# Save Model

In [44]:
saver = tf.train.Saver()

save_path = saver.save(sess, "model/model.ckpt")
print("Model saved in path: %s" % save_path)

Model saved in path: model/model.ckpt


In [56]:
import os
import tensorflow as tf

trained_checkpoint_prefix = 'model/model.ckpt'
export_dir = os.path.join('export_model')

graph = tf.Graph()
with tf.compat.v1.Session(graph=graph) as sess:
    # Restore from checkpoint
    loader = tf.compat.v1.train.import_meta_graph(trained_checkpoint_prefix + '.meta')
    loader.restore(sess, trained_checkpoint_prefix)

    # Export checkpoint to SavedModel
    builder = tf.compat.v1.saved_model.builder.SavedModelBuilder(export_dir)
    builder.add_meta_graph_and_variables(sess,
                                         [tf.saved_model.TRAINING, tf.saved_model.SERVING],
                                         strip_default_attrs=True)
    builder.save()

INFO:tensorflow:Restoring parameters from model/model.ckpt
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export_model/saved_model.pb


In [54]:
from tensorflow import keras
import tensorflowjs as tfjs

def importModel(modelPath):
    model = keras.models.load_model(modelPath)
    tfjs.converters.save_keras_model(model, "tfjsmodel")

importModel("export_model")

IndexError: list index (0) out of range

# Reset for the Next Run

In [ ]:
train_writer.close()
validation_writer.close()
sess.close()
tf.reset_default_graph()